### Importing Bibs

In [1]:
# append src path to modules
import sys
import os
sys.path.append("../") # go to parent dir

# import sub-modules
import src.constants as co
import src.data.utils as du
import src.features.utils as fu
import src.models.utils as mu
import src.visualization.utils as vu

# further libraries
# dealing with package warnings
import warnings
#progress bar
from tqdm import tqdm

# working with tabular data
import pandas as pd
# working with arrays
import numpy as np
# plotting
import matplotlib.pyplot as plt

# supress warning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

Import form src.constants is working
Import form src.data.utils is working
Import form src.features.utils is working
Import form src.models.utils is working
Import form src.visualization.utils is working


### Constants

In [2]:
#--------- handle the folders and files ---------#
FIG_DIR = "D:\\work-hsu\\08_code_programms\\ai-in-production-example-development\\use-case-2\\reports\\figures"

#--------- file constants related to JSON Object and dictionary---------#
# file items 
FILE_NAME = "FILE_NAME"
FILE_PATH = "FILE_PATH"
FILE_TYPE = "FILE_TYPE"
FILE_FOLDER = "FILE_FOLDER"
FILE_DEVICES = "FILE_DEVICES"
# device items
SAMPLE_TIME = "SAMPLE_TIME"
CHANNEL_DESCRIPTIONS = "CHANNEL_DESCRIPTIONS"
SAMPLES = "SAMPLES"
# sample items
EC_SYSTEM_TIMESTAMP = "EC_SYSTEM_TIMESTAMP"
CHANNEL_DATA = "CHANNEL_DATA"
SCALED_CHANNEL_DATA = "SCALED_CHANNEL_DATA"
# setable lables
WEIGHT_LABEL = "WEIGHT_LABEL"
FREQUENCY_LABEL = "FREQUENCY_LABEL"
MOVING_LABEL = "MOVING_LABEL"
# device name
DEVICE = "DEVICE"
DEVICE_DATA_FRAME = "DEVICE_DATA_FRAME"

### Converting raw data into data file

In [ ]:
RAW_DATA_DIR = "D:\\work-hsu\\08_code_programms\\ai-in-production-example-development\\use-case-2\\data\\raw"
PROCESSED_DATA_DIR = "D:\\work-hsu\\08_code_programms\\ai-in-production-example-development\\use-case-2\\data\\processed"

DATA_FILE_NAME = "raw_data_frame"

# list of files with JSON data
files_list = du.parse_files(RAW_DATA_DIR, recursive=True)

#--------- create the dictionary with raw data ---------#
data_dict = du.create_data_dict(files_list)

# creating the data frames and Extend Dictionaries
print("Creating Data Frames:")
for file_dict in tqdm(data_dict.values(), desc="Data Frame"):
    #crreate and insert dataframe of device data (time series data)
    for device_dict in file_dict[FILE_DEVICES].values():
        du.create_dict_item(device_dict, DEVICE_DATA_FRAME, du.create_data_frame(device_dict))
    # create and append weight label (kg)
    du.create_dict_item(file_dict, WEIGHT_LABEL, du.get_weight_label(file_dict[FILE_FOLDER]))
    # create and append freq label (herz)
    du.create_dict_item(file_dict, FREQUENCY_LABEL, du.get_frequency_label(file_dict[FILE_FOLDER]))
    # create and append moving label (up down)
    for device_dict in file_dict[FILE_DEVICES].values():
        #device_data_frame = create_data_frame(device_dict)
        moving_label = du.get_moving_label(device_dict[DEVICE_DATA_FRAME], device_dict[DEVICE_DATA_FRAME].columns[3])
    du.create_dict_item(file_dict, MOVING_LABEL, moving_label)

#--------- features ---------#
print("Creating Feature Frames:")
feature_frame = du.create_feature_frame(data_dict)

#example for selection of data of the frame
#test_select = feature_frame[feature_frame["MOVING_LABEL"]=="up"]

du.save_file(feature_frame, PROCESSED_DATA_DIR, DATA_FILE_NAME, "pickle")

### Printing the raw data

In [ ]:
DATA_FILE_PATH = f"D:\\work-hsu\\08_code_programms\\ai-in-production-example-development\\use-case-2\\data\processed\\{DATA_FILE_NAME}.pickle"

# load data frame for printing
data_frame = du.load_file(DATA_FILE_PATH)

# selecting data were mass moves upwards and weights 1710kg
select_data = data_frame[(data_frame[du.MOVING_LABEL]=="up") & (data_frame[du.WEIGHT_LABEL]== 1710)]

FIG_WIDTH = vu.cm2in(16)
FIG_HEIGHT = vu.cm2in(6)


# begin the first printing figure
fig1, ax1 = plt.subplots(1,1, figsize=(FIG_WIDTH ,FIG_HEIGHT))

# figure were both axis are in different graphs
fig2, (ax2, ax3) = plt.subplots(1,2, figsize=(FIG_WIDTH ,FIG_HEIGHT), tight_layout=True)

for index, row in tqdm(select_data.iterrows(), desc="Creating Plot"):
    column_names = row[du.DEVICE_DATA_FRAME].columns
    x_col = column_names[0]
    y_col = column_names[5]
    y_values = row[du.DEVICE_DATA_FRAME][y_col]
    x_values = row[du.DEVICE_DATA_FRAME][x_col].subtract(row[du.DEVICE_DATA_FRAME][x_col].iloc[0]).divide(1e9)
    # set the printing color
    if row[du.FREQUENCY_LABEL] == 60:
        temp_color = "r"
        temp_custom_label = "(bad)"
        vu.custom_plot(ax2, x_values, y_values, alpha = 0.3, color = temp_color, linewidth = 0.25, 
                        autoscale = False, x_limits=(0, 6), y_limits=(-0.5, 2), 
                        title="Messwerte unbearbeitet \n 1710kg (öffnen)", x_label= "Zeit in [1s]", y_label="Relativer Schleppfehler [1]",
                        plot_custom_legend = True, custom_label = temp_custom_label)
    if row[du.FREQUENCY_LABEL] == 69:
        temp_color = "g"
        temp_custom_label = "(good)"
        vu.custom_plot(ax3, x_values, y_values, alpha = 0.3, color = temp_color, linewidth = 0.25, 
                        autoscale = False, x_limits=(0, 6), y_limits=(-0.5, 2),
                        title = "Messwerte unbearbeitet \n 1710kg (öffnen)", x_label= "Zeit in [1s]", y_label="Relativer Schleppfehler [1]",
                        plot_custom_legend = True, custom_label = temp_custom_label)

    vu.custom_plot(ax1, x_values, y_values, alpha = 0.3, color = temp_color, linewidth = 0.25,
                    title = "Messwerte unbearbeitet \n 1710kg (öffnen)", x_label="Zeit in [1s]", y_label="Relativer Schleppfehler [1]",
                    plot_custom_legend=True, custom_label=temp_custom_label)

vu.save_fig(fig1, FIG_DIR, "raw_common", overwrite=True)
vu.save_fig(fig2, FIG_DIR, "raw_seperate", overwrite=True)

fig1.show()
fig2.show()

### Preprocess the Data

In [ ]:
#file name
FEATURE_FILE_NAME = "feature_frame"

# settings of the preprocessing
RESAMPLETIME = '0.05s'
MIN_ELEMENTS = 0
MAX_ELEMENTS = 100

data_frame = du.load_file(DATA_FILE_PATH)
#resample and cutting
drop_list = []
for index, row in data_frame.iterrows():
        try:
            # if any None, NaN etc. than raise the exception
            if row["DEVICE_DATA_FRAME"].isnull().values.any() == True:
                raise
            # convert to datetime
            row["DEVICE_DATA_FRAME"]["EC_SYSTEM_TIMESTAMP"] = pd.to_datetime(row["DEVICE_DATA_FRAME"]["EC_SYSTEM_TIMESTAMP"], unit = "ns")
            # resample 
            row["DEVICE_DATA_FRAME"] = row["DEVICE_DATA_FRAME"].resample(RESAMPLETIME, on="EC_SYSTEM_TIMESTAMP").mean()
            # reindexing 
            row["DEVICE_DATA_FRAME"].reset_index(inplace=True)
            # convert with pos interval
            row["DEVICE_DATA_FRAME"] = row["DEVICE_DATA_FRAME"][(row["DEVICE_DATA_FRAME"]['Aktueller Wert / DigitalausgÃ¤nge GrundgerÃ¤t ']==1)]
            # cut to 100
            row["DEVICE_DATA_FRAME"] = row["DEVICE_DATA_FRAME"][MIN_ELEMENTS:MAX_ELEMENTS]
            # reconvert to unix time
            row["DEVICE_DATA_FRAME"]["EC_SYSTEM_TIMESTAMP"] = row["DEVICE_DATA_FRAME"]["EC_SYSTEM_TIMESTAMP"].apply(lambda x: (x - pd.Timestamp("1970-01-01"))//pd.Timedelta("1ms"))

        except:
            # name dropping element
            print(f"Error with data row {index}, name: {row.FILE_NAME}")
            drop_list.append(f"{row.FILE_NAME}")
            print("Appended to drop list")

# drop elements of dataframe

for drop_element in drop_list:
    data_frame.drop(data_frame[data_frame.FILE_NAME == drop_element].index, inplace = True)
    print(f"Element {drop_element} dropped")

print(f"Number of elements that had been dropped: {len(drop_list)}")
# file saving
du.save_file(data_frame, PROCESSED_DATA_DIR, FEATURE_FILE_NAME, "pickle")

### Printing preprocessed Data

In [ ]:
FEATURE_PATH= f"D:\\work-hsu\\08_code_programms\\ai-in-production-example-development\\use-case-2\\data\processed\\{FEATURE_FILE_NAME}.pickle"

FIG_WIDTH = vu.cm2in(16)
FIG_HEIGHT = vu.cm2in(7)

data_frame = du.load_file(FEATURE_PATH)

# selecting data were mass moves upwards and weights 1710kg
select_data = data_frame[(data_frame[du.MOVING_LABEL]=="up") & (data_frame[du.WEIGHT_LABEL]== 1710)]

# begin the first printing figure
fig1, ax1 = plt.subplots(1,1, figsize=(FIG_WIDTH, FIG_HEIGHT))

# figure were both axis are in different graphs
fig2, (ax2, ax3) = plt.subplots(1,2, figsize=(FIG_WIDTH, FIG_HEIGHT), tight_layout=True)

for index, row in tqdm(select_data.iterrows(), desc="Creating Plot 1"):
    column_names = row[du.DEVICE_DATA_FRAME].columns
    x_col = column_names[0]
    y_col = column_names[5]
    y_values = row[du.DEVICE_DATA_FRAME][y_col]
    x_values = row[du.DEVICE_DATA_FRAME][x_col].subtract(row[du.DEVICE_DATA_FRAME][x_col].iloc[0]).divide(1e3)
    # set the printing color
    if row[du.FREQUENCY_LABEL] == 60:
        temp_color = "r"
        temp_custom_label = "(bad)"
        vu.custom_plot(ax2, x_values, y_values, alpha = 0.5, color = temp_color, linewidth = 0.25, 
                        autoscale = False, x_limits=(0, 5), y_limits=(-0.5, 2), 
                        title="Technikum BBG Messwerte vorverarbeitet \n 1710kg (open)", x_label= "Zeit in [1s]", y_label="Relativer Schleppfehler [-]",
                        plot_custom_legend = True, custom_label = temp_custom_label)
    if row[du.FREQUENCY_LABEL] == 69:
        temp_color = "g"
        temp_custom_label = "(good)"
        vu.custom_plot(ax3, x_values, y_values, alpha = 0.5, color = temp_color, linewidth = 0.25, 
                        autoscale = False, x_limits=(0, 5), y_limits=(-0.5, 2),
                        title = "Technikum BBG Messwerte vorverarbeitet \n 1710kg (open)", x_label= "Zeit in [1s]", y_label="Relativer Schleppfehler [-]",
                        plot_custom_legend = True, custom_label = temp_custom_label)

    vu.custom_plot(ax1, x_values, y_values, alpha = 0.5, color = temp_color, linewidth = 0.25,
                    title = "Technikum BBG Messwerte vorverarbeitet \n 1710kg (open)", x_label= "Zeit in [1s]", y_label="Relativer Schleppfehler [-]",
                    plot_custom_legend=True, custom_label=temp_custom_label)

vu.save_fig(fig1, FIG_DIR, "processed_common", overwrite=True)
vu.save_fig(fig2, FIG_DIR, "processed_seperate", overwrite=True)

fig1.show()
fig2.show()

### Train Model

In [7]:
#import numpy as np
#from src.data.utils import load_file, MOVING_LABEL, WEIGHT_LABEL

FEATURE_FILE_NAME = "feature_frame"
FEATURE_PATH= f"D:\\work-hsu\\08_code_programms\\ai-in-production-example-development\\use-case-2\\data\processed\\{FEATURE_FILE_NAME}.pickle"
MODEL_PATH = "D:\\work-hsu\\08_code_programms\\ai-in-production-example-development\\use-case-2\\src\\models"

data = du.load_file(FEATURE_PATH)

#upwards data witch 1.71t
select_data = data[(data[MOVING_LABEL]=="up") & (data[WEIGHT_LABEL]== 1710)]
select_data.reset_index(inplace=True)

VALUE_NAME = "Schleppfehler in Anwendereinheiten / Regelverfahren Prozesswerte"
AXIS_0 = len(select_data)
AXIS_1 = len(select_data["DEVICE_DATA_FRAME"].iloc[0][VALUE_NAME])

print(f"Number of Training Elements: {AXIS_0}")
X = np.zeros((AXIS_0,AXIS_1))
Y = np.zeros(AXIS_0)

for index, row in select_data.iterrows():
    X[index] = np.array(row["DEVICE_DATA_FRAME"][VALUE_NAME], dtype=float)

    label = np.array(row["FREQUENCY_LABEL"], dtype=int)
    if label == 60:
            # good label
            Y[index] = 1
    elif label == 69:
            # bad label
            Y[index] = 0
    else:
        print("Label Error")

#model creation
model = mu.create_model()

# model training
history = mu.train_model(model, X, Y, X, Y, epochs=100 , batch_size=100)

mu.save_model(model=model, file_path=MODEL_PATH, model_name="first_test_model")

#vu.custom_fig_multi()
#fig1.show()

Number of Training Elements: 67


d:\work-hsu\08_code_programms\ai-in-production-example-development\use-case-2\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "my_sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,121 (43.44 KB)

 Trainable params: 11,121 (43.44 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 810ms/step - accuracy: 0.3284 - loss: 0.7642
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.3284 - loss: 0.7494
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3284 - loss: 0.7345
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.3284 - loss: 0.7188
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3284 - loss: 0.7015
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.3433 - loss: 0.6842
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7164 - loss: 0.6704
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.9552 - loss: 0.6585
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.8358 - loss: 0.6479
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.7463 - loss: 0.6377
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6716 - loss: 0.6288
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6716 - l

### Prediction

In [15]:
MODEL_PATH = "D:\\work-hsu\\08_code_programms\\ai-in-production-example-development\\use-case-2\\src\models\\first_test_model.pickle"

#loading the model
model = mu.load_model(file_path=MODEL_PATH)

prediction_results = model.predict(X)
real_label = Y

for pred, label in zip(prediction_results, real_label):
    print(f"Real Label: {label} -> Prediction {int(pred*100)}%")


model loaded from path:{file_path}
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Real Label: 1.0 -> Prediction 87%
Real Label: 1.0 -> Prediction 98%
Real Label: 1.0 -> Prediction 89%
Real Label: 1.0 -> Prediction 87%
Real Label: 1.0 -> Prediction 88%
Real Label: 1.0 -> Prediction 87%
Real Label: 1.0 -> Prediction 70%
Real Label: 1.0 -> Prediction 85%
Real Label: 1.0 -> Prediction 90%
Real Label: 1.0 -> Prediction 73%
Real Label: 1.0 -> Prediction 93%
Real Label: 1.0 -> Prediction 93%
Real Label: 1.0 -> Prediction 89%
Real Label: 1.0 -> Prediction 97%
Real Label: 1.0 -> Prediction 78%
Real Label: 1.0 -> Prediction 85%
Real Label: 1.0 -> Prediction 92%
Real Label: 1.0 -> Prediction 98%
Real Label: 1.0 -> Prediction 90%
Real Label: 1.0 -> Prediction 92%
Real Label: 1.0 -> Prediction 96%
Real Label: 1.0 -> Prediction 89%
Real Label: 0.0 -> Prediction 3%
Real Label: 0.0 -> Prediction 6%
Real Label: 0.0 -> Prediction 6%
Real Label: 0.0 -> Prediction 2%
Real Label: 0.0 -> Prediction 6%
Real Label: 0.

C:\Users\Schieseck\AppData\Local\Temp\ipykernel_43368\1416043747.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Real Label: {label} -> Prediction {int(pred*100)}%")
